In [1]:
:opt no-lint

# Self-interpretation in the $\lambda$-caculus
참고자료: http://www.cs.ioc.ee/ewscs/2016/geuvers/geuvers-slides-lecture2.pdf

매우 간단하지만 모든 계산이 가능한 람다계산법에서
람다계산법 자신에 대한 인터프리터를 작성하는 방법에 대해 알아보는
2016년 에스토니아 컴퓨터과학 겨울학교 강의자료를 따라가 보기로 하자.

이것을 프로그래밍 언어의 예로 들면 하스켈로 하스켈 인터프리터를 작성한다거나
파이썬으로 파이썬 인터프리터를 작성한다거나 하는 자기 자신의 언어에 대한
인터프리터를 바로 그 언어로 작성하는 것과 마찬가지이다. 이것을 가장 간단하지만
모든 프로그래밍 언어로 표현가능한 계산을 다 표현할 수 있는 람다계산법으로 해보자는 것.

---

## 람다계산법에서 데이터의 인코딩

$ e \in \textsf{Exp} ~ ::= ~ x ~\mid~ (e_1~e_2) ~\mid~ (\lambda x.e) $

이렇게 간단한 3가지의 식으로만 이루어진 언어가 모든 계산을 다 할 수 있다고 한다.

일단 처음 람다계산법을 보면 이것이 쉽게 믿기지 않는다.
왜냐하면 람다계산법에는 데이터를 구성하는 방법이 따로 없고 함수만 만들 수 있기 때문이다.

보통 우리가 생각하는 계산은 어떤 데이터를 입력으로 받아 새로운 데이터를 결과값으로 계산하는 방식이다.
그러니까 우리가 생각하는 계산이란 계산의 대상이 되는 데이터 입력 데이터 $d$를
계산과정을 나타내는 함수(혹은 프로시저) $f$에 넘겨 실행하는 $f(d)$로 이해할 수 있다.

람다계산법에는 함수밖에 없기 때문에 데이터도 함수로 표현 혹은 함수로 인코딩해서 활용하는 방식으로
이런 우리가 생각하는 모든 계산을 진행할 수 있다고 알려져 있다. 람다계산법이 원래 함수를 표현하기
위해 만들어졌으므로 $f$를 인코딩할 수 있으리라는 것은 것은 누구나 상식적이라 받아들인다.
그렇다면 데이터 $d$를 어떤 식으로 인코딩하는지가 문제로 남는다.

람다식으로 인코딩한다는 의미로 $\lceil \cdot \rceil$라는 표기법을 쓰겠다.
 * $\lceil f\, \rceil$는 $f$를 인코딩한 람다식 
 * $\lceil d\, \rceil$는 $d$를 인코딩한 람다식

그러니까 $\lceil f\, \rceil\,\lceil d\, \rceil$는 $f(d)$를 인코딩한 람다식에 해당한다.
이렇게 인코딩할 수 있다면 $f(d)$를 계산하는 대신
$\lceil f\, \rceil\,\lceil d\, \rceil$를 람다식의 계산규칙에 따라 기계적으로 계산하면 된다.

### 진리값 데이터의 인코딩
0개나 1개의 값만으로 구성되는 보통 Void나 Unit이라고 부르는 데이터도 있지만 이건 너무 단순한 경우니까 넘어가자.

그 다음으로 가장 간단한 경우인 2개의 값으로 이루어지는 보통 Bool이라고 부르는 진리값 데이터를 인코딩해 보자.

즉 $\lceil\texttt{True}\rceil$와 $\lceil\texttt{False}\rceil$를 각각 적절한 람다식에 대응시켜 보자는 이야기.

그냥 하스켈의 람다식을 이용해서 람다계산법을 손쉽게 실행시키면서 작성해 보기로 하자.

In [2]:
true  = \x -> \y -> x  -- ⌈ True ⌉ = λx.λy.x
false = \x -> \y -> y  -- ⌈ False⌉ = λx.λy.y

In [3]:
true -- 람다식 즉 함수는 보통 프로그래밍 언어에서 안을 직접 들여다볼 수 없어서 일반적으로 출력되지 않음

: 

In [4]:
-- 인코딩된 진리값 람다식을 디코딩, 그러니까 ⌈ b ⌉ 로부터 b를 뽑아내는 함수
-- (람다식은 함수라서 하스켈에서 출력해 주지 않으므로 결과를 출력해 검사하기 위한 용도)
decodeBool b = b True False

In [5]:
decodeBool true

True

In [6]:
decodeBool false

False

In [7]:
-- if b then e1 else e0 를 인자를 3개 연달아 받는 함수로 생각해 이를 람다식으로 인코딩
_if_then_else = \b -> \e1 -> \e0 -> b e1 e0

In [8]:
_if_then_else true  "e1" "e0"

"e1"

In [9]:
_if_then_else false "e1" "e0"

"e0"

In [10]:
_not = \b -> \x -> \y -> b y x  -- λb.λx.λy.b y x 

In [13]:
_not true -- 결과값도 함수라서 출력 안됨

: 

In [11]:
decodeBool $ _not true

False

In [12]:
decodeBool $ _not false

True

In [14]:
_and = \b1 -> \b2 -> b1 (b2 true false) false -- λb1.λb2.b1 (b2 ⌈ True ⌉ ⌈ False⌉) ⌈ False⌉

In [16]:
_and true true -- 결과값도 함수라서 출력 안됨

: 

In [15]:
decodeBool $ _and true true

True

In [17]:
decodeBool $ _and true false

False

In [18]:
decodeBool $ _and false true

False

In [19]:
decodeBool $ _and false false

False

----

In [ ]:
{-# LANGUAGE TemplateHaskell #-}
import Control.Monad
import Language.Haskell.TH

_U :: Int -> Int -> Q Exp
_U n i = do
  xs <- replicateM n (newName "x")
  let args = map VarP xs
  return $ LamE args (VarE $ xs!!(i-1))

In [ ]:
$(_U 2 1) "e1" "e2"  -- λx.λy.x
$(_U 2 2) "e1" "e2"  -- λx.λy.x

In [ ]:
$(_U 3 1) "e1" "e2" "e3"  -- λx.λy.λz.x
$(_U 3 2) "e1" "e2" "e3"  -- λx.λy.λz.y
$(_U 3 3) "e1" "e2" "e3"  -- λx.λy.λz.z

In [ ]:
{-# LANGUAGE ImpredicativeTypes #-}
_Var :: d ->      (forall e.(forall a b c.a -> b -> c -> a) -> d -> e -> d)      -> d
_App :: d -> d -> (forall e.(forall a b c.a -> b -> c -> b) -> d -> d -> e -> d) -> d
_Abs :: d ->      (forall e.(forall a b c.a -> b -> c -> c) -> d -> e -> d)      -> d

_Var = \x e   -> e $(_U 3 1) x e
_App = \x y e -> e $(_U 3 2) x y e
_Abs = \x e   -> e $(_U 3 3) x e